In [ ]:
# Import packages
from getpass import getpass
import logging
import os
from pathlib import Path
import re
import time
from IPython.display import display, Image
import flywheel

In [ ]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

In [ ]:
project_id = '5eafb9fd788701016978097d'
fw = flywheel.Client('')
project = fw.get_project(project_id)

In [ ]:
def upload_file_to_acquistion(acquistion, fp, update=True, **kwargs):
    """Upload file to Acquisition container and update info if `update=True`
    
    Args:
        acquisition (flywheel.Acquisition): A Flywheel Acquisition
        fp (Path-like): Path to file to upload
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.        
    """
    basename = os.path.basename(fp)
    if not os.path.isfile(fp):
        raise ValueError(f'{fp} is not file.')
        
    if acquistion.get_file(basename):
        log.info(f'File {basename} already exists in container. Skipping.')
        return
    else:
        log.info(f'Uploading {fp} to acquisition {acquistion.id}')
        acquistion.upload_file(fp)
        while not acquistion.get_file(basename):   # to make sure the file is available before performing an update
            acquistion = acquistion.reload()
            time.sleep(1)
            
    if update and kwargs:
        f = acquisition.get_file(basename)
        f.update(**kwargs)
        


In [ ]:
PATH_TO_DATA = Path('/')


In [ ]:
files_to_upload = list(PATH_TO_DATA.rglob('fs.zip'))
dl = '\n'
print(f'Files to upload: \n{dl.join(map(str, files_to_upload))}')

In [ ]:
files_to_upload

subjects_id=[p.parts[5] for p in files_to_upload]

In [ ]:
def get_or_create_subject(project, label, update=True, **kwargs):
    """Get the Subject container if it exists, else create a new Subject container.
    
    Args:
        project (flywheel.Project): A Flywheel Project.
        label (str): The subject label.
        update (bool): If true, update container with key/value passed as kwargs.
        kwargs (dict): Any key/value properties of subject you would like to update.

    Returns:
        (flywheel.Subject): A Flywheel Subject container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    subject = project.subjects.find_first(f"label='{label}'")
    if not subject:
        subject = project.add_subject(label=label)
        
    if update and kwargs:
        subject.update(**kwargs)

    if subject:
        subject = subject.reload()

    return subject

In [ ]:
 def get_or_create_acquisition(session, label, update=True, **kwargs):  
    """Get the Acquisition container if it exists, else create a new Acquisition container.
    
    Args:
        session (flywheel.Session): A Flywheel Session.
        label (str): The Acquisition label.
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.

    Returns:
        (flywheel.Acquisition): A Flywheel Acquisition container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    acquisition = session.acquisitions.find_first(f'label={label}')
    if not acquisition:
        acquisition = session.add_acquisition(label=label)
        
    if update and kwargs:
        acquisition.update(**kwargs)

    if acquisition:
        acquisition = acquisition.reload()

    return acquisition


In [ ]:
def upload_file_to_acquistion(acquistion, fp, update=True, **kwargs):
    """Upload file to Acquisition container and update info if `update=True`
    
    Args:
        acquisition (flywheel.Acquisition): A Flywheel Acquisition
        fp (Path-like): Path to file to upload
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.        
    """
    basename = os.path.basename(fp)
    if not os.path.isfile(fp):
        raise ValueError(f'{fp} is not file.')
        
    if acquistion.get_file(basename):
        log.info(f'File {basename} already exists in container. Skipping.')
        return
    else:
        log.info(f'Uploading {fp} to acquisition {acquistion.id}')
        acquistion.upload_file(fp)
        while not acquistion.get_file(basename):   # to make sure the file is available before performing an update
            acquistion = acquistion.reload()
            time.sleep(1)
            
    if update and kwargs:
        f = acquisition.get_file(basename)
        f.update(**kwargs)


In [ ]:
log.info('Starting upload...')
for subj in subjects_id:
    log.info('Processing subject %s', str(subj))
    subject = get_or_create_subject(project, subj, update=False,)  # passing some value for the sake of the example
    print(f'{subject.sessions}')
#     #for ses in subj.glob('anx*'):
#         log.info('Processing session %s', str(ses))
#         session = get_or_create_session(subject, ses.name)
#         for acq in ses.glob('T1*'):            
#             log.info('Processing acquisition %s', str(acq))            
#             acquisition = get_or_create_acquisition(session, acq.name)
#             for file in acq.glob('*.dcm.zip'):
#                 upload_file_to_acquistion(acquisition, file)
# log.info('DONE')

In [ ]:
project.subjects

In [ ]:
project.subjects.find("label='18112'")

In [ ]:
# Hard code the session to pick if there are multiple sessions for a subject
subject_to_session_map = {
    # The syntax here is subject_label: session_label
    "18112": "18112",
}

In [ ]:
subjects_in_project = project.subjects()
print(f"{len(subjects_in_project)} Subjects found")
for subject in subjects_in_project:
    if subject.label in subjects_id:
        print(f"\tsubject label = {subject.label}")
        
        if (len(subject.sessions()) > 1):
            session = [ses for ses in subject.sessions() if ses.label == subject_to_session_map[subject.label]][0]
        else:
            session = subject.sessions()[0]
            
        print(f"\t\tsession label = {session.label}")
        t1_acq = [acq for acq in session.acquisitions() if acq.label == "t1"][0]
        print(f"\t\t\tFound acquisition {t1_acq.label}")